# World Battle Network

Like the marriage and kill networks the battle network joins up dynasties where a member of one has killed a member of the other. While India is still isolated the other dynasties form interesting patterns and relationships. While wars and battles are common deaths in battle aren't.

In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [2]:
client = MongoClient()
characters = client.ck2.characters

## Kill Network

In [3]:
pipeline = [
    {
        "$match" : {"c_d" : "death_battle"}
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"        
    },    
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    #{
    #    "$match" : {"$or" : [ {"dynasty.culture" : "irish"}, {"killer_dyn.culture" : "irish"}, {"culture" : "irish"}, {"killer_data.culture" : "irish"}] }
    #},
    {"$project" : {"name" : "$bn", "dynasty" : "$dynasty._id", "killer" : "$killer_data.bn", "killer_dynasty" : "$killer_dyn._id"}},
    #{"$limit" : 20}
]

In [4]:
karl = characters.aggregate(pipeline)

In [5]:
kill_df = pd.DataFrame(list(karl))

## Get all Dynasties involved

In [6]:
total_dyns = set(kill_df['dynasty'].unique())
total_dyns = total_dyns.union(set(kill_df['killer_dynasty'].unique()))
total_dyns_as_ints = [int(i) for i in list(total_dyns)]

In [7]:
dynasties = client.ck2.dynasties

pipeline = [    
    {
        "$match" : {"_id" : {"$in" : total_dyns_as_ints}}
    },
    {
        "$project" : {"name" : 1, "religion" : 1, "culture" : 1}
    },
    {
        "$sort" : {"name" : 1}
    }
]

In [8]:
dyns = dynasties.aggregate(pipeline)

# Build a Network Graph

In [9]:
import networkx as nx
import matplotlib.pyplot as plt

In [10]:
G = nx.Graph()

for dyn in dyns:
    if "name" in dyn and "religion" in dyn and "culture" in dyn: #needs to be added if building the full graph, a family with no name gets in somehow
        G.add_node(dyn["_id"], name = dyn['name'], culture = dyn['culture'], religion = dyn['religion'])

In [11]:
#complete_set = set()

for i in range(len(kill_df)):
    if G.has_edge(kill_df.loc[i, "killer_dynasty"], kill_df.loc[i, "dynasty"]):
        G.edge[kill_df.loc[i, "killer_dynasty"]][kill_df.loc[i, "dynasty"]]["weight"] +=1
    else:
        G.add_edge(kill_df.loc[i, "killer_dynasty"], kill_df.loc[i, "dynasty"], weight = 1)

G.remove_nodes_from(nx.isolates(G)) #drop unconnected nodes       

In [12]:
nx.write_graphml(max(nx.connected_component_subgraphs(G), key=len), "ck2-Battle-Kill-Network.graphml")

The graphml file in the code above was opened in Gephi and the picture below was generated. The nodes are colored by religion. India is in the top right and the green nodes are Hindu.  The blue nodes are Islamic and the orange are Catholic. The large pink nodes in the bottom right is the Baltic Pagen Penikis dynasty which conquers Finland and Rus. 

In [13]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://www.anquantarbuile.com/static/images/ck2/BattleNetworkReligion.png")